In [1]:
import warnings  
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics

In [33]:
#import csv
df = pd.read_csv('curriculum_access_data2.csv')

#drop duplicate index
df.drop(columns = 'Unnamed: 0', axis = 1, inplace = True)

#rename values

df.loc[df['program_id'] == 1.0, 'program_id'] = 'Web Development'
df.loc[df['program_id'] == 2.0, 'program_id'] = 'Web Development'
df.loc[df['program_id'] == 4.0, 'program_id'] = 'Web Development'         
df.loc[df['program_id'] == 3.0, 'program_id'] = 'Data Science'

#add network 1 & 2 columns and host 1 & 2 to split ip address

df[['network1','network2', 'host1', 'host2']] = df.ip.str.split(".", expand = True)

df['network'] = df['network1'] + df['network2']
df['host'] = df['host1'] + df['host2']

#drop network1 & 2, host 1 & 2
df = df.drop(columns=['network1', 'network2', 'host1', 'host2'])

df = df.rename(columns = {'name': 'cohort'})

#new column from date and time
df['access'] = df['date'] + ' ' + df['time']

#covert time to datetime w/ time only
df['time'] = pd.to_datetime(df['time'], format= '%H:%M:%S' ).dt.time

#drop time columns
#df = df.drop(columns = ['time'])

#convert to datetime 
df.access = pd.to_datetime(df.access)

#convert date column to datetime
df.date = pd.to_datetime(df.date)

#convert start_date to datetime
df.start_date = pd.to_datetime(df.start_date)

#convert end_date to datetime
df.end_date = pd.to_datetime(df.end_date)

#set date as index, sort index
#df = df.set_index('date').sort_index()

#drop paths that only have / in it
df = df[df.path != '/']

#drop staff
df = df[df.cohort != 'Staff']

df.head()

,date,time,path,user_id,ip,cohort,start_date,end_date,program_id,network,host,access
1,2018-01-26,09:56:02,java-ii,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,Web Development,97105,1961,2018-01-26 09:56:02
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,Web Development,97105,1961,2018-01-26 09:56:05
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,Web Development,97105,1961,2018-01-26 09:56:06
4,2018-01-26,09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,Web Development,97105,1961,2018-01-26 09:56:24
5,2018-01-26,09:56:41,javascript-i/loops,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,Web Development,97105,1961,2018-01-26 09:56:41


In [37]:
df.shape

(772219, 12)

In [34]:
#convert time column to time only
#df['time'] = pd.to_datetime(df['time'],format= '%H:%M:%S' ).dt.time
#df.head()

In [81]:
#create df to look atcounts/use_id

new = df.groupby('user_id')['date', 'path', 'ip' ,'access'].nunique()
new.head()

,date,path,ip,access
user_id,,,,
1,152,247,9,1392
2,83,150,12,1462
3,104,168,25,1492
4,52,142,11,633
5,81,196,4,1588


In [82]:
#reset the index to obtain user_id column
new = new.reset_index()

In [83]:
#create function to return outlier free df and outlier df to examine

def discard_outliers(df, k, col_list):
    
    for col in col_list:
        #obtain quartiles
        q1, q3 = df[col].quantile([.25, .75]) 
        
        #obtain iqr range
        iqr = q3 - q1
        
        upper_bound = q3 + k * iqr
        lower_bound = q1 - k * iqr
        
        
#         if q1 - (k * iqr) < 0:
#             lower_bound = 4
            
#         else:
#             lower_bound = q1 - (k * iqr)
#                 #high_outlier = q3 + (k * iqr)


        #return outlier - free df
        df_norm = df[(df[col] > lower_bound) & (df[col] < upper_bound)]
        
        #df_out = df[(df[col] < lower_bound) & (df[col] >= upper_bound)]

    return df_norm

In [84]:
col_list = ['date', 'path', 'ip' ,'access']
df_norm = discard_outliers(new, 1.5, col_list)
df_norm.head()

,user_id,date,path,ip,access
0,1,152,247,9,1392
1,2,83,150,12,1462
2,3,104,168,25,1492
3,4,52,142,11,633
4,5,81,196,4,1588


In [85]:
df_norm.describe()

,user_id,date,path,ip,access
count,926.000000,926.000000,926.000000,926.000000,926.000000
mean,483.892009,69.613391,113.835853,7.093952,733.870410
std,283.901679,51.340997,66.443270,10.886581,621.373224
min,1.000000,1.000000,1.000000,1.000000,1.000000
25%,237.250000,21.000000,53.000000,2.000000,173.250000
50%,477.500000,74.000000,138.500000,4.000000,626.500000
75%,728.750000,107.000000,163.000000,8.000000,1129.750000
max,981.000000,305.000000,348.000000,148.000000,2616.000000


In [86]:
#only keep dates > 4 and access > 49 and paths > 9 

df_norm = df_norm.loc[df_norm['date'] > 4]
df_norm = df_norm.loc[df_norm['access'] > 49]
df_norm = df_norm.loc[df_norm['path'] > 9]


df_norm.describe()

,user_id,date,path,ip,access
count,795.000000,795.000000,795.000000,795.000000,795.00000
mean,497.672956,80.571069,130.210063,8.008805,850.96478
std,276.875939,47.118516,56.713673,11.488848,593.88309
min,1.000000,5.000000,11.000000,1.000000,50.00000
25%,268.500000,37.000000,80.000000,3.000000,367.50000
50%,501.000000,85.000000,145.000000,5.000000,752.00000
75%,732.500000,112.000000,167.000000,9.000000,1228.50000
max,979.000000,305.000000,348.000000,148.000000,2616.00000


In [87]:
df.user_id.nunique()

947

In [88]:
df_norm.user_id.nunique()

795

In [76]:
#only_outliers = pd.merge(df, temp_df['user_id'], how = 'inner', on = 'user_id')

In [89]:
no_outliers = pd.merge(df, df_norm['user_id'], how = 'inner', on = 'user_id')

no_outliers.shape

(692888, 12)

In [71]:
no_outliers.user_id.nunique()

795

In [72]:
no_outliers.head()

,date,time,path,user_id,ip,cohort,start_date,end_date,program_id,network,host,access
0,2018-01-26,09:56:02,java-ii,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,Web Development,97105,1961,2018-01-26 09:56:02
1,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,Web Development,97105,1961,2018-01-26 09:56:05
2,2018-01-26,09:56:06,slides/object_oriented_programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,Web Development,97105,1961,2018-01-26 09:56:06
3,2018-01-26,10:40:15,javascript-i/functions,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,Web Development,97105,1961,2018-01-26 10:40:15
4,2018-01-26,11:26:13,java-i,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,Web Development,97105,1961,2018-01-26 11:26:13


In [50]:
no_outliers.columns

Index(['date', 'time', 'path', 'user_id', 'ip', 'cohort', 'start_date',
       'end_date', 'program_id', 'network', 'host', 'access'],
      dtype='object')

In [90]:
df_norm.describe()

,user_id,date,path,ip,access
count,795.000000,795.000000,795.000000,795.000000,795.00000
mean,497.672956,80.571069,130.210063,8.008805,850.96478
std,276.875939,47.118516,56.713673,11.488848,593.88309
min,1.000000,5.000000,11.000000,1.000000,50.00000
25%,268.500000,37.000000,80.000000,3.000000,367.50000
50%,501.000000,85.000000,145.000000,5.000000,752.00000
75%,732.500000,112.000000,167.000000,9.000000,1228.50000
max,979.000000,305.000000,348.000000,148.000000,2616.00000


In [63]:
no_outliers.groupby('user_id')['date', 'path', 'ip' ,'access'].nunique().describe()

,date,path,ip,access
count,794.000000,794.000000,794.000000,794.000000
mean,80.656171,130.340050,8.015113,851.973552
std,47.087040,56.630795,11.494712,593.575487
min,5.000000,11.000000,1.000000,51.000000
25%,37.000000,80.000000,3.000000,369.000000
50%,85.000000,145.000000,5.000000,753.000000
75%,112.000000,167.000000,9.000000,1229.250000
max,305.000000,348.000000,148.000000,2616.000000


In [64]:
df.groupby('user_id')['date', 'path', 'ip' ,'access'].nunique().describe()

,date,path,ip,access
count,947.000000,947.000000,947.000000,947.000000
mean,72.579725,116.006336,7.262936,795.076030
std,55.674613,67.514355,10.983628,762.696817
min,1.000000,1.000000,1.000000,1.000000
25%,23.000000,54.000000,2.000000,175.500000
50%,77.000000,139.000000,4.000000,637.000000
75%,109.000000,165.000000,8.000000,1173.000000
max,480.000000,348.000000,148.000000,8085.000000


In [55]:
out = no_outliers[['user_id']]
out

,user_id
0,1
1,1
2,1
3,1
4,1
...,...
692833,975
692834,975
692835,975
692836,975


In [ ]:
k = 1.5

 #obtain quartiles
q1, q3 = df[col].quantile([.25, .75]) 
        
        #obtain iqr range
iqr = q3 - q1
        
upper_bound = q3 + k * iqr
lower_bound = q1 - k * iqr

In [ ]:

for col in col_list:
        #obtain quartiles
        q1, q3 = df[col].quantile([.25, .75]) 
        
        #obtain iqr range
        iqr = q3 - q1
        
        upper_bound = q3 + k * iqr
        lower_bound = q1 - k * iqr
        

In [58]:
def get_bounds(df, k, col):
    
        #obtain quartiles
        q1, q3 = df[col].quantile([.25, .75]) 
        
        #obtain iqr range
        iqr = q3 - q1
        
        upper_bound = q3 + k * iqr
        lower_bound = q1 - k * iqr
        
        print(upper_bound)
        print(lower_bound)

In [59]:
get_bounds(new, 1.5, 'date')

238.0
-106.0


In [60]:
get_bounds(new, 1.5, 'access')

2669.25
-1320.75


In [61]:
get_bounds(new, 1.5, 'path')

331.5
-112.5


In [98]:
get_bounds(new, 1.5, 'ip')

17.0
-7.0


In [91]:
#IQR w/minimums

temp_df = new.loc[(new['date'] < 5) | (new['date'] > 238 )]
temp_df = new.loc[(new['access'] < 50) | (new['access'] > 2669)]
temp_df = new.loc[(new['path'] < 10) | (new['path'] > 331)]


temp_df.shape

(60, 5)

In [92]:
new.shape

(947, 5)

In [95]:
temp_df.describe()

,user_id,date,path,ip,access
count,60.000000,60.000000,60.00000,60.000000,60.000000
mean,380.416667,7.166667,10.80000,1.383333,51.933333
std,302.521230,39.146906,44.35481,1.208608,335.803356
min,62.000000,1.000000,1.00000,1.000000,1.000000
25%,113.250000,1.000000,3.00000,1.000000,3.000000
50%,268.000000,1.000000,5.00000,1.000000,7.000000
75%,578.750000,3.000000,8.00000,1.000000,12.000000
max,981.000000,305.000000,348.00000,8.000000,2609.000000


In [96]:
only_outliers = pd.merge(df, temp_df['user_id'], how = 'inner', on = 'user_id')
only_outliers.shape

(3613, 12)

In [97]:
only_outliers.head(20)

,date,time,path,user_id,ip,cohort,start_date,end_date,program_id,network,host,access
0,2018-01-29,13:08:57,java-ii,62,70.123.209.40,NaN,NaT,NaT,NaN,70123,20940,2018-01-29 13:08:57
1,2018-01-29,13:09:01,java-iii,62,70.123.209.40,NaN,NaT,NaT,NaN,70123,20940,2018-01-29 13:09:01
2,2018-01-29,13:09:14,spring,62,70.123.209.40,NaN,NaT,NaT,NaN,70123,20940,2018-01-29 13:09:14
3,2018-01-30,15:01:14,content/html-css,71,173.173.116.10,Quincy,2017-06-05,2017-09-22,Web Development,173173,11610,2018-01-30 15:01:14
4,2018-01-30,15:01:21,content/git,71,173.173.116.10,Quincy,2017-06-05,2017-09-22,Web Development,173173,11610,2018-01-30 15:01:21
5,2018-01-30,15:01:28,content/git/creating-a-repository.html,71,173.173.116.10,Quincy,2017-06-05,2017-09-22,Web Development,173173,11610,2018-01-30 15:01:28
6,2018-01-30,15:01:40,content/git/initializing-a-repository.html,71,173.173.116.10,Quincy,2017-06-05,2017-09-22,Web Development,173173,11610,2018-01-30 15:01:40
7,2018-01-31,19:10:00,prework/fundamentals,77,131.194.98.179,Glacier,2015-06-05,2015-10-06,Web Development,131194,98179,2018-01-31 19:10:00
8,2018-01-31,19:10:09,prework/databases,77,131.194.98.179,Glacier,2015-06-05,2015-10-06,Web Development,131194,98179,2018-01-31 19:10:09
9,2018-01-31,19:10:15,prework/databases/database-systems,77,131.194.98.179,Glacier,2015-06-05,2015-10-06,Web Development,131194,98179,2018-01-31 19:10:15


In [99]:
#create df to look at counts/use_id
new = df.groupby('user_id')['date', 'path', 'ip' ,'access'].nunique()

#reset the index to obtain user_id column
new = new.reset_index()

#IQR df w/minimums bc of negatives
temp_df = new.loc[(new['date'] < 5) | (new['date'] > 238 )]
temp_df = new.loc[(new['access'] < 50) | (new['access'] > 2669)] #access = date + time
temp_df = new.loc[(new['path'] < 10) | (new['path'] > 331)]
temp_df = new.loc[(new['ip'] < 2) | (new['ip'] > 17)]

only_outliers = pd.merge(df, temp_df['user_id'], how = 'inner', on = 'user_id')
only_outliers.shape

(146999, 12)

In [100]:
only_outliers.user_id.nunique()

252

In [101]:
947-252

695

In [102]:
695/947

0.73389651531151